## Session 02: Optimization of the Linear Regression Model

### Goran S. Milovanović, Phd

Feedback should be send to [goran.milovanovic@datakolektiv.com](mailto:goran.milovanovic@datakolektiv.com). 

![](DK_ML_ConfPalic2023.png)

### Lecturers

[Goran S. Milovanović, PhD, DataKolektiv, Chief Scientist & Owner](https://www.linkedin.com/in/gmilovanovic/)

[Aleksandar Cvetković, PhD, DataKolektiv, Consultant](https://www.linkedin.com/in/alegzndr/)

[Ilija Lazarević, MA, DataKolektiv, Consultant](https://www.linkedin.com/in/ilijalazarevic/)

![](DK_Logo_100.png)

![](../img/DK_Logo_100.png)

***

## Setup

In [ ]:
# - lib
import os
import numpy as np
import pandas as pd
import scipy
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import statsmodels.api as sm
import statsmodels.formula.api as smf

# - parameters
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
sns.set_theme()
# - rng
rng = np.random.default_rng()
# - plots
plt.rc("figure", figsize=(8, 6))
plt.rc("font", size=14)
sns.set_theme(style='white')

# - directory tree
data_dir = os.path.join(os.getcwd(), '_data')

## 0. Covariance and Correlation

### Covariance

Given two random variables (RVs), $X$ and $Y$, their (sample) covariance is given by:

$$cov(X,Y) = E[(X-E[X])(Y-E[Y])] = \frac{(X-\bar{X})(Y-\bar{Y})}{N-1}$$

where $E[X]$ denotes the *expectation* (the *mean*, if you prefer) of $X$, $\bar{X}$ is the mean of $X$, $\bar{Y}$ is the mean of $Y$, and $N$ is the sample size.


**Some properties of covariance.**

**Bilinearity.** Covariance is a bilinear function, meaning that it is linear in each argument separately. That is, for any constants a and b, we have:

$$cov(aX+b, Y) = acov(X,Y) + bcov(Y,Y)$$

and

$$cov(X, aY+b) = acov(X,Y) + bcov(X,X)$$

**Symmetry.** Covariance is a symmetric measure, meaning that the covariance between X and Y is the same as the covariance between Y and X. That is:

$$cov(X,Y) = cov(Y,X)$$

**Independence.** If X and Y are independent random variables, then their covariance is zero. That is:

$$cov(X,Y) = E[(X-\bar{X})(Y-\bar{Y})] = E[X-\bar{X}]E[Y-\bar{Y}] = 0$$

The **covariance of a variable with itself is equal to its variance**.

Python:

In [ ]:
def covariance(x, y):
    if len(x) != len(y):
        raise ValueError("x and y must be of same size.")
    # the means of x and y
    x_mean = np.mean(x)
    y_mean = np.mean(y)    
    # the deviations of x and y from their means
    x_dev = x - x_mean
    y_dev = y - y_mean
    # the covariance of x and y
    cov = np.mean(x_dev * y_dev)
    # ouput
    return cov

In [ ]:
sample1 = norm.rvs(loc=1.7, scale=.6, size=1000)
sample2 = norm.rvs(loc=2.85, scale=.6, size=1000)
covariance(sample1, sample2)

In [ ]:
sample1 = norm.rvs(loc=1.7, scale=.6, size=100000)
# linear scaling of sample1
a, b = .75, 2
sample2 = a*sample1+b
# add random noise to sample2
eps = norm.rvs(loc=0, scale=0.27, size=100000)
sample2 = sample2 + eps
samples = pd.DataFrame({'x':sample1, 'y':sample2})
samples.plot.scatter(x='x', y='y')

In [ ]:
covariance(samples['x'], samples['y'])

### Correlation

**Pearson's coefficient of correlation** is nothing else than a covariance between $X$ and $Y$ upon their *standardization*. The standardization of a RV - widely known as a variable *z-score* - is obtained upon subtracting all of its values from the mean, and dividing by the standard deviation; for the **i**-th observation of $X$

$$z(x_i) = \frac{x_i-\bar{X}}{\sigma}$$

Python:

In [ ]:
def z_score(x):
    # the mean and standard deviation of x
    x_mean = np.mean(x)
    x_std = np.std(x)    
    # the z-score of x
    z = (x - x_mean) / x_std
    # output
    return z

In [ ]:
samples

In [ ]:
samples['x_z'] = z_score(samples['x'])
samples['y_z'] = z_score(samples['y'])
samples

In [ ]:
print(np.mean(samples['x_z']))
print(np.std(samples['x_z'], ddof=1))
print(np.mean(samples['y_z']))
print(np.std(samples['y_z'], ddof=1))

In [ ]:
covariance(samples['x_z'], samples['y_z'])

In [ ]:
from scipy.stats import pearsonr
pearsonr(samples['x_z'], samples['y_z'])

Right. There are many formulas that compute `r`, the correlation coefficient; however, understanding that is simply the covariance of standardized RVs is essential. Once you know to standardize the variables and how to compute covariance (and that is easy), you don't need to care about expressions like:

$$r_{XY} = \frac{N\sum{XY}-(\sum{X})(\sum{Y})}{\sqrt{[N\sum{X^2}-(\sum{X})^2][N\sum{Y^2}-(\sum{Y})^2]}}$$

This and similar expressions are good, and especially for two purposes: first, they will compute the desired value of the correlation coefficient in the end, that's for sure, and second, writing them up in really helps mastering $\LaTeX$. Besides these roles they play, there is really nothing essentially important in relation to them.

Somewhat easier to remember:

$$r_{XY} = \frac{cov(X,Y)}{\sigma(X)\sigma(Y)}$$

i.e. correlation is the covariance of $X$ and $Y$, divided by the product of their standard deviations.

## 1. Simple Linear Regression

In [ ]:
### --- Setup - importing the libraries

# - supress those annoying 'Future Warning'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# - data
import numpy as np
import pandas as pd

# - os
import os

# - ml
import statsmodels.api as sm
import statsmodels.formula.api as smf


# - visualization
import matplotlib.pyplot as plt
import seaborn as sns

# - parameters
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
sns.set_theme()
# - rng
rng = np.random.default_rng()
# - plots
plt.rc("figure", figsize=(8, 6))
plt.rc("font", size=14)
sns.set_theme(style='white')

# - directory tree
data_dir = os.path.join(os.getcwd(), '_data')

We will use the `Fish.csv` data set in this session. You can grab it from [Kaggle: fish-market](https://www.kaggle.com/datasets/aungpyaeap/fish-market). Please place the `Fish.csv` data set into your `_data` directory.

In [ ]:
# - loading the dataset
# - Kaggle: https://www.kaggle.com/datasets/aungpyaeap/fish-market
# - place it in your _data/ directory
fish_data = pd.read_csv(os.path.join(data_dir, 'Fish.csv'))
fish_data.head(10)

In [ ]:
fish_data.info()

### Target: predict Weight from Height

In [ ]:
model_frame = fish_data[['Height', 'Weight']]
model_frame

In [ ]:
# - fitting the linear model to the data
linear_model = smf.ols(formula='Weight ~ Height', data=model_frame).fit()
linear_model.summary()

Linear model has the form

$$y = \beta_1 x + \beta_0 + \varepsilon,$$

where
- $y$ - the true value of the *target variable*
- $\beta_1$ - the *slope* of the model
- $\beta_0$ - the *intercept* of the model
- $\varepsilon$ - the *residual*

The predicted value $\hat{y}$ of the target variable is computed via Liner regression via

$$\hat{y} = \beta_1 x + \beta_0.$$

In [ ]:
# - model's parameters; Height represent the slope k
linear_model.params

In [ ]:
# - predicting the value using model's formula and parameters
model_frame['Predicted Values'] = linear_model.predict()

In [ ]:
# - calculating the residuals - the difference between the true and predicted values
model_frame['Residuals'] = linear_model.resid

In [ ]:
model_frame

In [ ]:
# - some statistics on the residuals
model_frame['Residuals'].describe()

In [ ]:
# - plotting the true data, predicted values and the prediction line
sns.regplot(data=model_frame, x='Height', y='Weight', ci=0, line_kws={'color':'red'})
sns.scatterplot(data=model_frame, x='Height', y='Predicted Values', color='red', s=50)
sns.despine(offset=10, trim=True)
plt.title('Weight ~ Hight', fontsize=14);

Ok, `statsmodels` can do it; how do we find out about the optimal values of $\beta_0$ and $\beta_1$?
Let's build ourselves a function that (a) tests for some particular values of $\beta_0$ and $\beta_1$ for a particular regression problem (i.e. for a particular dataset) and returns the model error.

The model error? Oh. Remember the residuals:

$$\epsilon_i = y_i - \hat{y_i}$$

where $y_i$ is the observation to be predicted, and $\hat{y_i}$ the actual prediction?

Next we do something similar to what happens in the computation of variance, square the differences:

$$\epsilon_i^2 = (y_i - \hat{y_i})^2$$

and define the model error for all observations to be **the sum of squares**:

$$SSE = \sum_{i=1}^{N}(y_i - \hat{y_i})^2$$

Obviously, the lower the $SSE$ - the Sum of Squared Error - the better the model! Here's a function that returns the SSE for a given data set (with two columns: the predictor and the criterion) and a choice of parameters $\beta_0$ and $\beta_1$:

In [ ]:
# - sse function
def lg_sse(pars):
    # - pick up the parameters
    beta_0 = pars[0]
    beta_1 = pars[1]
    # - predict from parameters
    preds = beta_0+beta_1*fish_data['Height']
    # - compute residuals
    residuals = fish_data['Weight']-preds
    # - square the residuals
    residuals = residuals**2
    # - sum of squares
    residuals = residuals.sum()
    # - out:
    return residuals

Test `lg_sse()` now:

In [ ]:
pars = [-144.385971, 60.496351]
print(lg_sse(pars))

Check via `statsmodels`:

In [ ]:
(linear_model.resid**2).sum()

Method A. Random parameter space search

In [ ]:
beta_0 = np.random.uniform(low=-200, high=200, size=10000)
beta_1 = np.random.uniform(low=-200, high=200, size=10000)
random_pars = pd.DataFrame({'beta_0':beta_0, 'beta_1':beta_1})
random_pars.head()

In [ ]:
sse = []
for i in range(random_pars.shape[0]):
    pars = [random_pars['beta_0'][i],random_pars['beta_1'][i]]
    sse.append(lg_sse(pars))
random_pars['sse'] = sse
random_pars.sort_values('sse', ascending=True, inplace=True)
random_pars.head()

Check with `statsmodels`:

In [ ]:
linear_model.params

Not bad, how about 100,000 random pairs?

In [ ]:
beta_0 = np.random.uniform(low=-200, high=200, size=100000)
beta_1 = np.random.uniform(low=-200, high=200, size=100000)
random_pars = pd.DataFrame({'beta_0':beta_0, 'beta_1':beta_1})
sse = []
for i in range(random_pars.shape[0]):
    pars = [random_pars['beta_0'][i],random_pars['beta_1'][i]]
    sse.append(lg_sse(pars))
random_pars['sse'] = sse
random_pars.sort_values('sse', ascending=True, inplace=True)
random_pars.head()

Method B. Grid search

In [ ]:
beta_0_vals = np.linspace(-200,200,100)
beta_1_vals = np.linspace(-200,200,100)
grid = np.array([(beta_0, beta_1) for beta_0 in beta_0_vals for beta_1 in beta_1_vals])
grid = pd.DataFrame(grid)
grid = grid.rename(columns={0: "beta_0", 1: "beta_1"})
grid.head()

In [ ]:
sse = []
for i in range(grid.shape[0]):
    pars = [grid['beta_0'][i],grid['beta_1'][i]]
    sse.append(lg_sse(pars))
grid['sse'] = sse
grid.sort_values('sse', ascending=True, inplace=True)
grid.head()

A grid more dense:

In [ ]:
beta_0_vals = np.linspace(-200,200,1000)
beta_1_vals = np.linspace(-200,200,1000)
grid = np.array([(beta_0, beta_1) for beta_0 in beta_0_vals for beta_1 in beta_1_vals])
grid = pd.DataFrame(grid)
grid = grid.rename(columns={0: "beta_0", 1: "beta_1"})
sse = []
for i in range(grid.shape[0]):
    pars = [grid['beta_0'][i],grid['beta_1'][i]]
    sse.append(lg_sse(pars))
grid['sse'] = sse
grid.sort_values('sse', ascending=True, inplace=True)
grid.head()

Check with `statsmodels`:

In [ ]:
linear_model.params

Method C. Optimization (the real thing)

The Method of Least Squares

Here is the real thing. 

- **Question.** What do we really need to do to find the optimal $\beta_0$, $\beta_1$ parameters of the Simple Linear Regression Model?
- **Answer.** Of course, we need to find $\beta_0$, $\beta_1$ **at the minimum of the $SSE$ - the error - function.**

And how do we do that?

Well, in a particular case of a (Simple or Multiple) Linear Regression Model, it turns out that is possible to provide an analytical solution for all model parameteres that minimize the model $SSE$ (error) function. It takes some time work through the partial derivates of $SSE$ in respect to each model parameter, but it works in the end.

*But finding analytical solutuion will not work for just any statistical model.*

Now, imagine that we have an algorithm - call it an **optimization algorithm ** - that can find the parameters that minimize a respective function. Indeed we have such an algorithm. Indeed we have many different such algorithms, developed by experts in the very, very alive and complicated branch of mathematics called Optimization Theory. We will put one such algorithm - the famed Nelder-Mead Simplex Method - to work in order to minimize $SSE$ in respect to $\beta_0$, $\beta_1$.

In [ ]:
import scipy as sp

# - sse function
def lg_sse(pars, data):
    # - pick up the parameters
    beta_0 = pars[0]
    beta_1 = pars[1]
    # - predict from parameters
    preds = beta_0+beta_1*fish_data['Height']
    # - compute residuals
    residuals = fish_data['Weight']-preds
    # - square the residuals
    residuals = residuals**2
    # - sum of squares
    residuals = residuals.sum()
    # - out:
    return residuals

# - initial (random) parameter values
init_beta_0 = np.random.uniform(low=-15, high=15, size=1)
init_beta_1 = np.random.uniform(low=-15, high=15, size=1)
init_pars = [init_beta_0, init_beta_1]

# - optimize w. Nelder-Mead
optimal_model = sp.optimize.minimize(
    # - fun(parameters, args)
    fun=lg_sse,
    args = (fish_data), 
    x0 = init_pars, 
    method='Nelder-Mead')

# - optimal parameters
optimal_model.x

Check against `statsmodels`

In [ ]:
linear_model.params

Final value of the objective function (the model SSE, indeed):

In [ ]:
lg_sse(pars=optimal_model.x, data=fish_data)

Check against `statsmodels`

In [ ]:
linear_model.ssr

Error Surface Plot: The Objective Function

In [ ]:
beta_0_vals = np.linspace(-160,-120,200)
beta_1_vals = np.linspace(40,80,200)
grid = np.array([(beta_0, beta_1) for beta_0 in beta_0_vals for beta_1 in beta_1_vals])
grid = pd.DataFrame(grid)
grid = grid.rename(columns={0: "beta_0", 1: "beta_1"})
sse = []
for i in range(grid.shape[0]):
    pars = [grid['beta_0'][i],grid['beta_1'][i]]
    sse.append(lg_sse(pars, fish_data))
grid['sse'] = sse
grid.sort_values('sse', ascending=True, inplace=True)
grid.head()

This the function that we have minimized:

In [ ]:
# - import plotly
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

# - Mesh3d: Objective Function
fig = go.Figure(data=[go.Mesh3d(
    x=grid['beta_0'], 
    y=grid['beta_1'], 
    z=grid['sse'], 
    color='lightblue', 
    opacity=0.50)])
fig.update_layout(scene = dict(
                    xaxis_title='Beta_0',
                    yaxis_title='Beta_1',
                    zaxis_title='SSE'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

Back to statsmodels

In [ ]:
# - Pearson's correlation (R-value) coefficient and R^2
print(f"Pearson's correlation (R-value): {round(np.sqrt(linear_model.rsquared), 4)}")
print(f"Coefficient of determination (R^2): {round(linear_model.rsquared, 4)}")

In [ ]:
# - p-values of the model's parameters
print(f"p-values: \n{linear_model.pvalues}")

In [ ]:
# --- Predicting new data

predictions = pd.DataFrame(columns=['Height', 'Weight'])

# - sampling the new data from the normal distribution with the mean and std parameters taken from the original data
new_fish_height = rng.normal(loc=model_frame['Height'].mean(), scale=model_frame['Height'].std(), size=100)
# - clipping the negative values 
new_fish_height = np.clip(new_fish_height, a_min=0, a_max=np.infty)
predictions['Height'] = new_fish_height

# - predicting the heights on the new data using the linear model
new_fish_weight = linear_model.predict(predictions['Height'])

# - displaying the new data and the corresponding predictions
predictions['Weight'] = new_fish_weight
print(predictions)

In [ ]:
# - plotting the predictions
sns.lineplot(data=predictions, x='Height', y='Weight', color='red')
sns.scatterplot(data=predictions, x='Height', y='Weight', color='red', s=50)
sns.despine(offset=10, trim=True)
plt.title('Predictions', fontsize=14);

## 2. Multiple Linear Regression

The formula for the Multiple Linear Regression Model has the form

$$y = \beta_1x_1 + \beta_2x_2 + \cdots + \beta_kx_k + \beta_0 + \varepsilon,$$

where 

- $y$ - the true value of target variable
- $x_1, x_2, \ldots, x_k$ - the predictors' values
- $\beta_1, \beta_2, \ldots, \beta_k$ - model's parameters for the predictors
- $\beta_0$ - the intercept of the model
- $\varepsilon$ - the residual

To predict a value $\hat{y}$ of the target variable via Multiple Linear Regression, we use

$$\hat{y} = \beta_1x_1 + \beta_2x_2 + \cdots + \beta_kx_k + \beta_0.$$

In [ ]:
# --- Composing the fomula of the model

predictors = fish_data.columns.drop('Weight')
print(predictors)

# - right side of the formula
formula = ' + '.join(predictors)

# - left side of the formula
formula = 'Weight ~ ' + formula

formula

In [ ]:
# - fitting the linear model to the data
linear_model = smf.ols(formula=formula, data=fish_data).fit()
linear_model.summary()

In [ ]:
# - model's parameters
linear_model.params

In [ ]:
# - predicting the values using linear model's formula and parameters
model_frame['Predicted Weight'] = linear_model.predict()

In [ ]:
# - plotting the true values vs predicted values
# - the identity line (y=x) shows how good is the prediction - the closer the datapoint to the line, the better
sns.scatterplot(data=model_frame, x='Predicted Weight', y='Weight')
sns.lineplot(x=np.arange(-500, 2000), y=np.arange(-500, 2000), color='k')
sns.despine(offset=10, trim=True)
plt.title('Model Predictions \n Note: identity line', fontsize=14);

### Further Reading:

- [https://www.khanacademy.org/math/statistics-probability/probability-library](https://www.khanacademy.org/math/statistics-probability/probability-library)

- [Random variables | Statistics and probability | Math | Khan Academy]( https://www.khanacademy.org/math/statistics-probability/random-variables-stats-library)

- [Brandon Foltz, Statistics PL15 - Multiple Linear Regression Playlist](https://www.youtube.com/playlist?list=PLIeGtxpvyG-IqjoU8IiF0Yu1WtxNq_4z-)


DataKolektiv, 2022/23.

[hello@datakolektiv.com](mailto:goran.milovanovic@datakolektiv.com)

![](../img/DK_Logo_100.png)

<font size=1>License: [GPLv3](https://www.gnu.org/licenses/gpl-3.0.txt) This Notebook is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. This Notebook is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with this Notebook. If not, see http://www.gnu.org/licenses/.</font>